# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [29]:
df_receipt_summary = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
df_receipt_log = pd.concat([df_receipt_summary[['customer_id', 'amount']], df_receipt_summary.amount.apply(lambda x: math.log10(x))], axis=1)
df_receipt_log.head(10)

,customer_id,amount,amount
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
5,CS001211000025,456,2.658965
6,CS001212000027,448,2.651278
7,CS001212000031,296,2.471292
8,CS001212000046,228,2.357935
9,CS001212000070,456,2.658965


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [30]:
df_receipt_log = pd.concat([df_receipt_summary[['customer_id', 'amount']], df_receipt_summary.amount.apply(lambda x: math.log(x))], axis=1)
df_receipt_log.head(10)

,customer_id,amount,amount
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
5,CS001211000025,456,6.122493
6,CS001212000027,448,6.104793
7,CS001212000031,296,5.690359
8,CS001212000046,228,5.429346
9,CS001212000070,456,6.122493


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [20]:
df_product['profit'] = df_product['unit_price'] - df_product['unit_cost']
df_product.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
0,P040101001,4,401,40101,198.0,149.0,49.0
1,P040101002,4,401,40101,218.0,164.0,54.0
2,P040101003,4,401,40101,230.0,173.0,57.0
3,P040101004,4,401,40101,248.0,186.0,62.0
4,P040101005,4,401,40101,268.0,201.0,67.0
5,P040101006,4,401,40101,298.0,224.0,74.0
6,P040101007,4,401,40101,338.0,254.0,84.0
7,P040101008,4,401,40101,420.0,315.0,105.0
8,P040101009,4,401,40101,498.0,374.0,124.0
9,P040101010,4,401,40101,580.0,435.0,145.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [31]:
# 答えみた

df_tmp = df_product.copy()
df_tmp['unit_profit_rate'] = (df_tmp['unit_price'] - df_tmp['unit_cost']) / df_tmp['unit_price']
df_tmp['unit_profit_rate'].mean(skipna=True)

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [32]:
# 答えみた

df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.floor(x / 0.7))
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit,new_price,new_profit_rate
0,P040101001,4,401,40101,198.0,149.0,49.0,212.0,0.297170
1,P040101002,4,401,40101,218.0,164.0,54.0,234.0,0.299145
2,P040101003,4,401,40101,230.0,173.0,57.0,247.0,0.299595
3,P040101004,4,401,40101,248.0,186.0,62.0,265.0,0.298113
4,P040101005,4,401,40101,268.0,201.0,67.0,287.0,0.299652
5,P040101006,4,401,40101,298.0,224.0,74.0,320.0,0.300000
6,P040101007,4,401,40101,338.0,254.0,84.0,362.0,0.298343
7,P040101008,4,401,40101,420.0,315.0,105.0,450.0,0.300000
8,P040101009,4,401,40101,498.0,374.0,124.0,534.0,0.299625
9,P040101010,4,401,40101,580.0,435.0,145.0,621.0,0.299517
